# CMB-S4: DRAFT tool

## loading necessary python modules

In [5]:
%load_ext autoreload
%autoreload 2

#%pylab notebook
%matplotlib inline
from pylab import *
import os

import argparse, sys, numpy as np, scipy as sc, warnings, os
sys.path.append('modules/')
import tools

#import matplotlib.cbook
warnings.filterwarnings('ignore',category=RuntimeWarning)
#warnings.filterwarnings('ignore', category=DeprecationWarning)
#warnings.filterwarnings('ignore', category=matplotlib.cbook.mplDeprecation)

# set parameters

In [6]:
fsky = 0.57
fix_params = ['mnu']
prior_dict = {'tau': 0.007}
results_folder = 'results/'
ilc_pgm_name = 'get_ilc_weights_and_residuals.py'
fisher_pgm_name = 'get_fisher_withdelensing.py'
'''
ilc_opfname = '%s/s4_cmb_ilc.npy' %(results_folder)
fisher_opfname = '%s/s4_cmb_fisher.npy' %(results_folder)
'''
ilc_folder = 'ilc_outputs/20210506_with202102designtoolinputforpySM3sims_sedscalingfordust/s4_so_joint_configs/'
ilc_opfname = '%s/s4wide_single_chlat_plus_aso/s4wide_single_chlat_plus_aso_ilc_galaxy1_27-39-93-145-225-278_TT-EE_galmask2_AZ_for7years.npy' %(ilc_folder)
fisher_opfname = ilc_opfname.replace('ilc_outputs/', 'results/')


# Step1: Execute ILC code to get the residuals.
# Step2: Execute the fisher code with the ILC residuals.

In [7]:
#execute
cmd1 = 'python3 %s -opfname %s' %(ilc_pgm_name, ilc_opfname)
print('\n######')
print(cmd1)
##os.system(cmd1)
cmd2 = 'python3 %s -ilc_fname %s -opfname %s' %(fisher_pgm_name, ilc_opfname, fisher_opfname)
print('\n######')
print(cmd2)
#os.system(cmd2)
print('######')


######
python3 get_ilc_weights_and_residuals.py -opfname ilc_outputs/20210506_with202102designtoolinputforpySM3sims_sedscalingfordust/s4_so_joint_configs//s4wide_single_chlat_plus_aso/s4wide_single_chlat_plus_aso_ilc_galaxy1_27-39-93-145-225-278_TT-EE_galmask2_AZ_for7years.npy

######
python3 get_fisher_withdelensing.py -ilc_fname ilc_outputs/20210506_with202102designtoolinputforpySM3sims_sedscalingfordust/s4_so_joint_configs//s4wide_single_chlat_plus_aso/s4wide_single_chlat_plus_aso_ilc_galaxy1_27-39-93-145-225-278_TT-EE_galmask2_AZ_for7years.npy -opfname results/20210506_with202102designtoolinputforpySM3sims_sedscalingfordust/s4_so_joint_configs//s4wide_single_chlat_plus_aso/s4wide_single_chlat_plus_aso_ilc_galaxy1_27-39-93-145-225-278_TT-EE_galmask2_AZ_for7years.npy
######


# now read the Fisher matrix and get constraints

In [8]:
#now read the Fisher matrix and get constraints
which_spectra = 'delensed'
results_dict = np.load(fisher_opfname, allow_pickle = True).item()
cosmo_params = results_dict['cosmoParams']
fisher_mat_dict = results_dict['fisherGaussian'][0]
fisher_mat = fisher_mat_dict[which_spectra] * fsky

# fix params and add priors

In [9]:
#fix params and add priors
fisher_mat, cosmo_params = tools.fn_fix_params(fisher_mat, cosmo_params, fix_params)
cosmo_params = np.asarray(cosmo_params)
#add prior
fisher_mat = tools.fn_add_prior(fisher_mat, cosmo_params, prior_dict)
print(fisher_mat)

[[ 4.55424816e+07  2.75554403e+07 -6.95956515e+05 -3.82939312e+13
   1.67939323e+06  4.41536089e+06  7.74805249e+08]
 [ 2.75554403e+07  3.04416055e+09 -1.09654013e+07  2.81856323e+16
   9.21543371e+07 -1.23861650e+08 -1.77117213e+10]
 [-6.95956515e+05 -1.09654013e+07  7.17742910e+04 -2.10044114e+14
  -5.80000334e+05  8.36204893e+05  2.19953969e+08]
 [-3.82939312e+13  2.81856323e+16 -2.10044114e+14  9.01622387e+23
   1.95698524e+15 -3.78661315e+15 -6.81811550e+17]
 [ 1.67939323e+06  9.21543371e+07 -5.80000334e+05  1.95698524e+15
   5.28379550e+06 -8.26801065e+06 -1.86295786e+09]
 [ 4.41536089e+06 -1.23861650e+08  8.36204893e+05 -3.78661315e+15
  -8.26801065e+06  1.64026998e+07  2.84325151e+09]
 [ 7.74805249e+08 -1.77117213e+10  2.19953969e+08 -6.81811550e+17
  -1.86295786e+09  2.84325151e+09  4.15284575e+12]]


# print constraints

In [10]:
cov_mat = np.linalg.inv(fisher_mat)
cosmo_constraints = np.diag(cov_mat)**0.5

print('One sigma errors are listed below:\n')
for p, sigma in zip(cosmo_params, cosmo_constraints):
    print('\tParam = %s; %g' %(p, sigma))

One sigma errors are listed below:

	Param = omega_c_h2; 0.000638
	Param = omega_b_h2; 4.42088e-05
	Param = N_eff; 0.0336912
	Param = A_s; 1.80689e-11
	Param = n_s; 0.00286747
	Param = tau; 0.00472488
	Param = theta_s; 7.43757e-07
